# Face Recognition on Photo/in Realtime

### ```Nhận diện khuôn mặt trong ảnh bằng Dlib```

##### Import package/framework

In [1]:
from tkinter import *
import cv2
from tkinter import messagebox
import os
import shutil
import numpy as np 
from pathlib import Path
from PIL import ImageTk, Image
from tkinter.filedialog import askopenfile
import face_recognition
import argparse
import pickle
import time
import imutils
from imutils import paths
from tkinter import font, filedialog
from tkinter.ttk import *


In [23]:
class App(Frame):
    def __init__(self, master=None):
        super().__init__(master)

        self.defaultFont = font.nametofont("TkDefaultFont")
        self.defaultFont.configure(family = "Calibri", size = 16, weight = font.BOLD)
        self.place(relx = 0.5, rely = 0.5, anchor = "center")
        self.pack()

def open_file():
   file = filedialog.askopenfile(mode='r', filetypes=[('PNG', '*.png'), ('JPG', '*.jpg')])
   if file:
        global filepath
        filepath = os.path.abspath(file.name)
        filepath = filepath.replace("\\", "\\\\")
        img = show_picture(filepath)
        global opened_file
        opened_file = Label(frm, image = img)
        opened_file.grid(column=1, row=1)
        opened_file.image = img
        global cv_img
        cv_img = cv2.imread(filepath, cv2.IMREAD_ANYCOLOR)

def open_filepath():
    global folder
    folder = filedialog.askdirectory(title="select").replace("/", "\\\\")
    if folder:
        Label(frm, text = folder).grid(column=2, row=5, columnspan = 3)
        

def show_picture(path):
    img = ImageTk.PhotoImage((Image.open(path)).resize((400,300)))
    return img

def func_Train():
    imagePaths = list(paths.list_images("dataset"))

    # khởi tạo list chứa known encodings và known names (để các test images so sánh)
    # chứa encodings và tên của các images trong dataset
    knownEncodings = []
    knownNames = []

    # duyệt qua các image paths
    for (i, imagePath) in enumerate(imagePaths):
        # lấy tên người từ imagepath
        print("[INFO] processing image {}/{}".format(i+1, len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        # load image bằng OpenCV và chuyển từ BGR to RGB (dlib cần)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Đối với từng image phải thực hiện detect face, trích xuất face ROI và chuyển về encoding
        # trả về array of bboxes of faces, dùng dlib như bài face detection đó
        # model="cnn" chính xác hơn nhưng chậm hơn, "hog" nhanh hơn nhưng kém chính xác hơn
        boxes = face_recognition.face_locations(rgb, model='cnn')    

        # tính the facial embedding for the face
        # sẽ tính encodings cho mỗi face phát hiện được trong ảnh (có thể có nhiều faces)
        # Để lý tưởng trong ảnh nên chỉ có một mặt người của mình thôi
        encodings = face_recognition.face_encodings(rgb, boxes)  

        # duyệt qua các encodings
        # Trong ảnh có thể có nhiều faces, mà ở đây chỉ có 1 tên
        # Nên chắc chắn trong dataset ban đầu ảnh chỉ có một mặt người thôi nhé
        # Lý tưởng nhất mỗi ảnh có 1 face và có 1 encoding thôi
        for encoding in encodings:
            # lưu encoding và name vào lists bên trên
            knownEncodings.append(encoding)
            knownNames.append(name)

    # dump (lưu) the facial encodings + names vào ổ cứng
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}

    with open('Models/encodings.pickle', "wb") as f:
        f.write(pickle.dumps(data))

def func_NhanDien():
    data = pickle.loads(open("Models\encodings.pickle", "rb").read())      # loads - load từ file
    rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

    boxes = face_recognition.face_locations(rgb, model='cnn') #hog hoặc cnn
    encodings = face_recognition.face_encodings(rgb, boxes)

    names = []

    # duyệt qua các encodings của faces phát hiện được trong ảnh
    for encoding in encodings:
        # khớp encoding của từng face phát hiện được với known encodings (từ dataset)
        # so sánh list of known encodings và encoding cần check, sẽ trả về list of True/False xem từng known encoding có khớp với encoding check không
        # có bao nhiêu known encodings thì trả về list có bấy nhiêu phần tử
        # trong hàm compare_faces sẽ tính Euclidean distance và so sánh với tolerance=0.6 (mặc định), nhó hơn thì khớp, ngược lại thì ko khớp (khác người)
        matches = face_recognition.compare_faces(data["encodings"], encoding, 0.4)      # có thể điều chỉnh tham số cuối
        name = "Unrecognized"   #Nếu lớn hơn thì xem như không nhận ra, để tạm, đổi tên sau

        # Kiểm tra xem từng encoding có khớp với known encodings nào không,
        if True in matches:
            # lưu các chỉ số mà encoding khớp với known encodings (nghĩa là b == True)
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]

            # tạo dictionary để đếm tổng số lần mỗi face khớp
            counts = {}
            # duyệt qua các chỉ số được khớp và đếm số lượng 
            for i in matchedIdxs:
                name = data["names"][i]     # tên tương ứng known encoding khớp với encoding check
                counts[name] = counts.get(name, 0) + 1  # nếu chưa có trong dict thì + 1, có rồi thì lấy số cũ + 1

            # lấy tên có nhiều counts nhất (tên có encoding khớp nhiều nhất với encoding cần check)
            # có nhiều cách để có thể sắp xếp list theo value ví dụ new_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
            # nó sẽ trả về list of tuple, mình chỉ cần lấy name = new_dic[0][0]
            name = max(counts, key=counts.get)

        names.append(name)

    # Duyệt qua các bounding boxes và vẽ nó trên ảnh kèm thông tin
    # Nên nhớ recognition_face trả bounding boxes ở dạng (top, rights, bottom, left)
    for ((top, right, bottom, left), name) in zip(boxes, names):
        cv2.rectangle(cv_img, (left, top), (right, bottom), (255, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15

        cv2.putText(cv_img, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 1)

    cv2.imwrite(".\\recognize.png", cv_img)

    recognized_img = show_picture(".\\recognize.png")
    recognized = Label(frm, image = recognized_img)
    recognized.grid(column=1, row=1)
    recognized.image = recognized_img

def get_data():
    global getdata_form
    getdata_form = Toplevel(padx=10, pady=10)
    getdata_form.resizable(0, 0)
    getdata_form.title("Get data and train model")

    Label(getdata_form, text='Name:', width=7).grid(column=0, row=0, ipadx = 3, ipady = 1)
    global nameUser
    nameUser = Entry(getdata_form, width=30)
    nameUser.grid(column=1, row=0, ipadx = 3, ipady = 1)

    Label(getdata_form, text='ID:', width=7).grid(column=0, row=1, ipadx = 3, ipady = 1)
    global idUser
    idUser = Entry(getdata_form, width=30)
    idUser.grid(column=1, row=1, ipadx = 3, ipady = 1)

    Button(getdata_form, text='Get data', command=getID_Name).grid(column=0, row=2, ipadx = 5, ipady = 10, columnspan=2)


def getID_Name():
    name_ = nameUser.get()
    id_ = idUser.get()
    getdata_form.destroy()
    take_images(id_, name_)


def take_images(name_,id_):
    datasets = 'dataset'
    sub_data = str(name_)+ '-' + str(id_)
    path = os.path.join(datasets, sub_data)
    if not os.path.isdir(path):
        os.mkdir(path)

    # defining the size of images
    (width, height) = (130, 100)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)

    # The program loops until it has 50 images of the face.
    count = 1
    while count < 50:
        (_, im) = webcam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (width, height))
            cv2.imwrite('% s/% s.png' % (path, count), face_resize)
            count += 1

        cv2.imshow('Take pictures', im)
        key = cv2.waitKey(10)
        if key == 27:
            break
        
    cv2.destroyAllWindows()
    messagebox.showinfo("Data has been retrieved","Model đã ghi nhận dữ liệu của bạn thành công")


# Tạo ứng dụng
myapp = App()

myapp.master.title("Face recognition on picture")
myapp.master.minsize(1000, 400)
myapp.master.maxsize(1500, 800)

frm = Frame(myapp, padding = 24)
frm.grid()

haar_file = 'Models/haarcascade_frontalface_default.xml'
cv_img = cv2.imread('.\\alert.png', cv2.IMREAD_ANYCOLOR)

Button(frm, text="Click to import picture", command = open_file).grid(column = 1, row = 0, pady = 10, ipady = 4, ipadx = 16, columnspan=2)

Label(frm, text = "The place to display the input image", background = "white", anchor="center").grid(column=1, row=1, ipadx = 85, ipady = 137, columnspan=2)

Button(frm, text="Get Data", command = get_data).grid(column = 0, row = 4, pady = 16, ipady = 12, ipadx = 16)
Button(frm, text="Train model", command = func_Train).grid(column = 1, row = 4, pady = 16, ipady = 12, ipadx = 16)
Button(frm, text="Recognize", command = func_NhanDien).grid(column = 2, row = 4, pady = 16, ipady = 12, ipadx = 16)
Button(frm, text="Exit", command = myapp.master.destroy).grid(column = 3, row = 4, ipady = 12, ipadx = 16)

myapp.mainloop()

### ```Realtime Webcam Face Recognizer dùng Dlib```

In [ ]:
data = pickle.load(open('Models\encodings.pickle', "rb"))      # loads - load từ file

# Khởi tạo video stream và pointer to the output video file, để camera warm up một chút
video = cv2.VideoCapture(0)     # có thể chọn cam bằng cách thay đổi src
writer = None
time.sleep(1)
frame_rate = 2
prev = 0

while True:
    ret, frame = video.read()

    if not ret:
        break
    time_elapsed = time.time() - prev

    #Giảm fps
    if time_elapsed > 1.0/frame_rate:
        prev = time.time()

        # chuyển frame từ BGR to RGB, resize để tăng tốc độ xử lý
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(rgb, width=150)
        # hệ số scale từ ảnh gốc (frame) về rgb, tí phải dùng bên dưới
        r = frame.shape[1] / float(rgb.shape[1])

        # CŨng làm tương tự cho ảnh test, detect face, extract face ROI, chuyển về encoding
        # rồi cuối cùng là so sánh kNN để recognize
        boxes = face_recognition.face_locations(rgb, model='hog') #hog hoặc cnn
        encodings = face_recognition.face_encodings(rgb, boxes)

        # khởi tạo list chứa tên các khuôn mặt phát hiện được
        # nên nhớ trong 1 ảnh có thể phát hiện được nhiều khuôn mặt nhé
        names = []

        # duyệt qua các encodings của faces phát hiện được trong ảnh
        for encoding in encodings:
            # khớp encoding của từng face phát hiện được với known encodings (từ datatset)
            # so sánh list of known encodings và encoding cần check, sẽ trả về list of True/False xem từng known encoding có khớp với encoding check không
            # có bao nhiêu known encodings thì trả về list có bấy nhiêu phần tử
            # trong hàm compare_faces sẽ tính Euclidean distance và so sánh với tolerance=0.6 (mặc định), nhó hơn thì khớp, ngược lại thì ko khớp (khác người)
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            name = "Unrecognized"    # tạm thời vậy, sau này khớp thì đổi tên

            # Kiểm tra xem từng encoding có khớp với known encodings nào không,
            if True in matches:
                # lưu các chỉ số mà encoding khớp với known encodings (nghĩa là b == True)
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]

                # tạo dictionary để đếm tổng số lần mỗi face khớp
                counts = {}
                # duyệt qua các chỉ số được khớp và đếm số lượng 
                for i in matchedIdxs:
                    name = data["names"][i]     # tên tương ứng known encoding khiowps với encoding check
                    counts[name] = counts.get(name, 0) + 1  # nếu chưa có trong dict thì + 1, có rồi thì lấy số cũ + 1

                # lấy tên có nhiều counts nhất (tên có encoding khớp nhiều nhất với encoding cần check)
                # có nhiều cách để có thể sắp xếp list theo value ví dụ new_dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
                # nó sẽ trả về list of tuple, mình chỉ cần lấy name = new_dic[0][0]
                name = max(counts, key=counts.get)

            names.append(name)

        # Duyệt qua các bounding boxes và vẽ nó trên ảnh kèm thông tin
        # Nên nhớ recognition_face trả bounding boxes ở dạng (top, rights, bottom, left)
        for ((top, right, bottom, left), name) in zip(boxes, names):
            """ Do đang làm việc với rgb đã resize rồi nên cần rescale về ảnh gốc (frame), nhớ chuyển về int """
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15

            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)

        if writer is not None:  # tiếp tục ghi frame đã chèn bboxes, name vào
            writer.write(frame)

        """ Check xem có muốn hiển thị ra màn hình ko, mặc định là có"""
        if 1 > 0: #0 là false, 1 là true
            cv2.imshow("Image", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break 

video.release()
cv2.destroyAllWindows() 

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

### ```Realtime Webcam Face Recognizer dùng LBPH Face Recognizer trong OpenCV```

##### Khai báo các lớp và hàm

In [24]:
haar_file = 'Models/haarcascade_frontalface_default.xml'
global isCameraRunning
isCameraRunning = False

class App(Frame):
    def __init__(self, master=None):
        super().__init__(master)

        self.defaultFont = font.nametofont("TkDefaultFont")
        self.defaultFont.configure(family = "Calibri", size = 16, weight = font.BOLD)
        self.place(relx = 0.5, rely = 0.5, anchor = "center")
        self.pack()


def TestModel():
# Training the images saved in training image folder
    datasets = 'dataset'
    # Create a list of images and a list of corresponding names
    (images, labels, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(datasets):
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(datasets, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                images.append(cv2.imread(path, 0))
                labels.append(int(lable))
            id += 1
    (width, height) = (130, 100)

    # Create a Numpy array from the two lists above
    (images, labels) = [np.array(lis) for lis in [images, labels]]

    # OpenCV trains a model from the images
    # NOTE FOR OpenCV2: remove '.face'
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(images, labels)

    # Part 2: Use fisherRecognizer on camera stream
    face_cascade = cv2.CascadeClassifier(haar_file)
    global webcam
    webcam = cv2.VideoCapture(0)
    isCameraRunning = True
    def show_Camera():
        (_, im) = webcam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (width, height))
            # Try to recognize the face
            prediction = model.predict(face_resize)
            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)

            if prediction[1] < 100:
                cv2.putText(im, '% s - %.0f' %
                            (names[prediction[0]], prediction[1]), (x - 10, y - 10),
                            cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
            else:
                cv2.putText(im, 'Unrecognized',
                            (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                
        imgtk = ImageTk.PhotoImage((Image.fromarray(im).resize((400,300))))
        
        camera = Label(frm, image=imgtk)
        camera.grid(column=1, row=1, ipadx = 0, ipady = 0)
        
        camera.imgtk = imgtk
        camera.image = imgtk
        
        camera.after(50, show_Camera)

    show_Camera()

def get_data():
    global getdata_form
    getdata_form = Toplevel(padx=10, pady=10)
    getdata_form.resizable(0, 0)
    getdata_form.title("Get data and train model")

    Label(getdata_form, text='Name:', width=7).grid(column=0, row=0, ipadx = 3, ipady = 1)
    global nameUser
    nameUser = Entry(getdata_form, width=30)
    nameUser.grid(column=1, row=0, ipadx = 3, ipady = 1)

    Label(getdata_form, text='ID:', width=7).grid(column=0, row=1, ipadx = 3, ipady = 1)
    global idUser
    idUser = Entry(getdata_form, width=30)
    idUser.grid(column=1, row=1, ipadx = 3, ipady = 1)

    Button(getdata_form, text='Get data', command=getID_Name).grid(column=0, row=2, ipadx = 5, ipady = 10, columnspan=2)


def getID_Name():
    name_ = nameUser.get()
    id_ = idUser.get()
    getdata_form.destroy()
    take_images(id_, name_)


def take_images(name_,id_):
    datasets = 'dataset'
    sub_data = str(name_)+ '-' + str(id_)
    path = os.path.join(datasets, sub_data)
    if not os.path.isdir(path):
        os.mkdir(path)

    # defining the size of images
    (width, height) = (130, 100)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)

    # The program loops until it has 50 images of the face.
    count = 1
    while count < 50:
        (_, im) = webcam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (width, height))
            cv2.imwrite('% s/% s.png' % (path, count), face_resize)
            count += 1

        cv2.imshow('Take pictures', im)
        key = cv2.waitKey(10)
        if key == 27:
            break
        
    cv2.destroyAllWindows()
    messagebox.showinfo("Data has been retrieved","Model đã ghi nhận dữ liệu của bạn thành công")

def exitApp():
    if isCameraRunning == False:
        webcam.release()
    myapp.master.destroy()

##### Khởi tạo form

In [25]:
# Tạo ứng dụng
myapp = App()

myapp.master.title("Real-time Face Recognition")
myapp.master.minsize(1000, 400)
myapp.master.maxsize(1500, 800)

frm = Frame(myapp, padding = 24)
frm.grid()


cv_img = cv2.imread('.\\alert.png', cv2.IMREAD_ANYCOLOR)

Label(frm, text = "The place to display the webcam", background = "white", anchor="center").grid(column=1, row=1, ipadx = 85, ipady = 137)

Button(frm, text="Get data", command = get_data).grid(column = 0, row = 4, pady = 16, ipady = 12, ipadx = 16)
Button(frm, text="Train and recognize", command = TestModel).grid(column = 1, row = 4, pady = 16, ipady = 12, ipadx = 16)
Button(frm, text="Exit", command = exitApp).grid(column = 2, row = 4, ipady = 12, ipadx = 16)

myapp.mainloop()